# Cleaning the Met Open Access Dataset (Kaggle)

In [ ]:
# These packages can also be installed in the CLI with 'conda' commands:

install.packages("tidyverse", repos='http://cran.us.r-project.org')
install.packages("tm", repos='http://cran.us.r-project.org')

In [ ]:
# These packages couldn't be installed via conda, so they were installed from CRAN:

install.packages("fedmatch", repos='http://cran.us.r-project.org')
install.packages("fuzzyjoin", repos='http://cran.us.r-project.org')

In [ ]:
# Import Libraries

library(tidyverse)
library(fuzzyjoin)
library(fedmatch)
library(tm)
library(stringr)
library(stringi)

# Data Collection

## Phase 1: Gather & Import Datasets

In [2]:
# Import Data
met_objects <- read.csv('../kaggle_csv/met_objects.csv')
countries <- read.csv('../kaggle_csv/countries.csv')

In [3]:
# Display columm names in met_objects

colnames(met_objects)

[1] "Object_Number"           "Is_Highlight"           
 [3] "Is_Public_Domain"        "Object_ID"              
 [5] "Department"              "Object_Name"            
 [7] "Title"                   "Culture"                
 [9] "Period"                  "Dynasty"                
[11] "Reign"                   "Portfolio"              
[13] "Artist_Role"             "Artist_Prefix"          
[15] "Artist_Name"             "Artist_Display_Bio"     
[17] "Artist_Suffix"           "Artist_Alpha_Sort"      
[19] "Nationality"             "Artist_Begin_Date"      
[21] "Artist_End_Date"         "Object_Date"            
[23] "Object_Begin_Date"       "Object_End_Date"        
[25] "Object_Year"             "Medium"                 
[27] "Dimensions"              "Credit"                 
[29] "Geography_Type"          "City"                   
[31] "State"                   "County"                 
[33] "Country"                 "Region"                 
[35] "Sub_Region"              "Locale"                 
[37] "Locus"                   "Excavation"             
[39] "River"                   "Classification"         
[41] "Rights_and_Reproduction" "Link_Resource"          
[43] "Metadata_Date"           "Repository"

In [4]:
# Select Met data into DF

met_df <- met_objects %>%
    select(Object_Number, Object_Name, Title, 
           Department, Period, Object_Year, 
           Geography_Type, Culture, Nationality, Country, Region, Sub_Region, 
           Artist_Name, Medium, Classification, Dimensions, Credit)

## Credits: Data Sources

* 1. met_objects dataset adapted from The Metropolitan Museum of Art Open Access's data on Kaggle @ https://www.kaggle.com/datasets/metmuseum/the-metropolitan-museum-of-art-open-access
* 2. countries dataset adapted from Andrada's dataset, on Kaggle @ https://www.kaggle.com/datasets/andradaolteanu/country-mapping-iso-continent-region 

# Data Cleaning: Geography (Culture, Country, Nationality)
## Phase 1: Parse Existing Values

## **Met: Culture**

In [5]:
# Make string lowercase

culture_temp <- met_df$Culture

culture_temp = tolower(culture_temp)

## **Met: Country**

In [6]:
# Make Strings Lowercase

country_temp <- met_df$Country

country_temp = tolower(country_temp)

In [7]:
# Remove Stop Words

stop_words_country <- c("former", "modern", "possibly", "posssibly", "probably", "prbably", "present-day")

country_temp  =  removeWords(country_temp, stop_words_country) 

In [8]:
# Split Strings via Special Character Delimiters

country_temp_2 <- strsplit(country_temp, split="[|,/?:[(]+")

m = 1

while (m < length(country_temp)) {
       
    if ((is.na(country_temp_2[[m]][1]) | country_temp_2[[m]][1] == "") & length(country_temp_2[[m]]) == 1) {
        country_temp[m] = "unknown"
    }
    
    else if ((is.na(country_temp_2[[m]][1]) | country_temp_2[[m]][1] == "") & length(country_temp_2[[m]]) > 1) {
        country_temp[m] = country_temp_2[[m]][2]
    }
    
    else if (grepl("eastern mediterranean or egypt", country_temp_2[[m]][1])) {
        country_temp[m] <- str_replace(country_temp_2[[m]][1], "eastern mediterranean or ", "")
    }
    
    else if (grepl("democratic republic of", country_temp_2[[m]][1]) & !grepl("congo", country_temp_2[[m]][1])) {
        country_temp[m] <- str_replace(country_temp_2[[m]][1], "democratic republic of", "")
    }
    
    else if (grepl("republic of", country_temp_2[[m]][1]) & !grepl("congo", country_temp_2[[m]][1]) & !grepl("korea", country_temp_2[[m]][1])) {
        country_temp[m] <- str_replace(country_temp_2[[m]][1], "republic of", "")
    }
    
    else if (grepl("central or ", country_temp_2[[m]][1])) {
        country_temp[m] <- str_replace(country_temp_2[[m]][1], "central or ", "")
    }
    
    else if (grepl("central", country_temp_2[[m]][1]) & !grepl("african republic", country_temp_2[[m]][1])) {
        country_temp[m] <- str_replace(country_temp_2[[m]][1], "central", "")
    }
    
    else {
        country_temp[m] = country_temp_2[[m]][1]
    }
    
    m = m + 1
}


In [9]:
country_temp <- clean_strings(country_temp)

In [10]:
# Cardinal Directions & "or" / "and": Round 1

delim_country_andor <- function(cty_list, cty_temp) {
    n = 1

    while (n < length(cty_list)) {
        if ((is.na(cty_temp[[n]][1]) | cty_temp[[n]][1] == '') & length(cty_temp[[n]]) == 1) {
            cty_list[n] = 'unknown'
        }
        
        else if ((is.na(cty_temp[[n]][1]) | cty_temp[[n]][1] == '') & length(cty_temp[[n]]) > 1) {
            cty_list[n] = cty_temp[[n]][2]
        }
        
        else if (grepl("north|east|south|west|central", cty_temp[[n]][1]) & !grepl(" ", cty_temp[[n]][1])) {
            cty_list[n] = cty_temp[[n]][2]
        }

        else {
            cty_list[n] = cty_temp[[n]][1]
        }

        n = n + 1
    }
    
    return(cty_list)
}

country_temp_3a <- strsplit(country_temp, split=" or ")

country_temp = delim_country_andor(country_temp, country_temp_3a)

country_temp_3b = strsplit(country_temp, split=" and ")

country_temp = delim_country_andor(country_temp, country_temp_3b)


In [11]:
# Cardinal Directions: Round 2

country_temp_4 <- strsplit(country_temp, split=" ")

o = 1

while (o < length(country_temp)) {

    if (length(country_temp_4[[o]]) > 1) {
        
        if (country_temp_4[[o]][1] == "south" & country_temp_4[[o]][2] == "africa") {
            
            country_temp[o] == "south africa"
        }
        
        else if (country_temp_4[[o]][1] == "western" & country_temp_4[[o]][2] == "sahara") {
            
            country_temp[o] == "western sahara"
        } 
        
        else if (country_temp_4[[o]][1] == "south" & country_temp_4[[o]][2] == "korea") {
            
            country_temp[o] == "south korea"
        } 
        
         else if (country_temp_4[[o]][1] == "south" & country_temp_4[[o]][2] == "sudan") {
            
            country_temp[o] == "south sudan"
        } 
        
         else if (country_temp_4[[o]][1] == "south" & country_temp_4[[o]][2] == "georgia") {
            
            country_temp[o] == "south georgia and the south sandwich islands"
        } 
        
        else {
            
            z <- country_temp_4[[o]][!grepl("north|east|south|west|lower|nothern|sothern|region", country_temp_4[[o]])]
            country_temp[o] = paste(z, collapse=' ')
        }
    }
    
    o = o + 1
}


In [12]:
# Standardize Country Spellings

country_temp = str_replace_all(country_temp, c("alamania|alamanian" = "germany",
                                                  "arabian peninsula" = "saudi arabia",
                                                  "austrian" = "austria",
                                                  "azerbaijian" = "azerbaijan",
                                                    "belgian congo|dem rep congo|dem rep of congo|democratic republic of congo|democratic republic of the congo" = "democratic republic of congo",
                                                    "brazzaville|french congo|republic of the congo|republic of congo" = "republic of congo",
                                                    "britain|britian|british isles|england|scotland" = "united kingdom",
                                                    "bohemia" = "czech republic",
                                                    "bosnia herzegovina" = "bosnia and herzegovina",
                                                    "burmah|burma" = "myanmar",
                                                    "alexandria|byzantine egypt|excavated church of the archangel michael|fustat" = "egypt",
                                                    "byzantium|crete" = "greece",
                                                    "anatolia|antioch|constaninople|constantinople|lake van|nicomedia" = "turkey",
                                                    "carpathian basin" = "hungary",
                                                    "czechoslovakia" = "czech republic",
                                                    "krinjabo|c te d ivoire|cote d ivoire|cote d'ivoire" = "ivory coast",
                                                    "damascus|emesa|jazira" = "syria",
                                                    "darabjird|persia" = "iran",
                                                    "republic of timor leste|democratic republic of timor leste" = "timor leste",
                                                    "federated states of micronesia|nauru" = "micronesia",
                                                    "brussels|flanders|flemish" = "belgium",
                                                    "french dahomey|dahomey|republic of benin|benin" = "benin",
                                                    "early islamic palestine" = "palestine",
                                                    "european" = "europe",
                                                    "amman|gerasa" = "jordan",
                                                    "germanyn|niederbreisig|rhein" = "germany",
                                                    "gotland" = "sweden",
                                                    "republic of guinea bissau|guinea bissau" = "guinea bissau",
                                                    "haute garonne" = "france",
                                                    "mangareva|marquesas islands|tahiti" = "french polynesia",
                                                    "the marshall islands" = "marshall islands",
                                                    "la t ne ii" = "gaul",
                                                    "friesland|holland|netherlandsish|the netherlands|netherlandish" = "netherlands",
                                                    "caesarea|canaan|latin kingdom of jerusalem|the holy land|jerusalem|tiberias" = "israel",
                                                    "kosova" = "kosovo",
                                                    "the iberian peninsula|iberian peninsula|spain or" = "spain",
                                                    "italian|rome" = "italy",
                                                    "liberia ivory coast" = "liberia",
                                                    "sabah|sarawak" = "malaysia",
                                                    "new republic" = "republic",
                                                    "philippine islands|republic of the philippines|the philippines" = "philippines",
                                                    "caucasus region|caucasus|russia federation|russian federation|russian" = "russia",
                                                    "sulawesi|sumatra" = "indonesia",
                                                    "tirana" = "albania",
                                                    "tunesia" = "tunisia",
                                                    "uganda sudan" = "uganda",
                                                    "character 0|lisez et propagez nos annales|uncertain mint|unknown country" = "unknown",
                                                    "united sates|united states of america|unites states|u s a" = "united states",
                                                    "wallis" = "switzerland",
                                                    "zanzibar" = "tanzania"
                                                   ))


# Standardize names shorter than their replacements:
country_temp_5 <- country_temp

b <- 1
while (b < length(country_temp)) {
    if (is.na(country_temp_5[b])) {
        country_temp[b] = "unknown"
    }
    
    else if (country_temp_5[b] == "america" | country_temp_5[b] == "united state" | country_temp_5[b] == "us" | country_temp_5[b] == "usa") {
        country_temp[b] = "united states"
    }
    
    else if (country_temp_5[b]== "arabia") {
        country_temp[b] = "saudi arabia"
    }
    
    else if (country_temp_5[b]== "bosnia") {
        country_temp[b] = "bosnia and herzegovina"
    }
    
    else if (country_temp_5[b]== "congo") {
        country_temp[b] = "republic of congo"
    }
    
    else if (country_temp_5[b] == "french") {
        country_temp[b] = "france"
    }
    
    else if (country_temp_5[b] == "german") {
        country_temp[b] = "germany"
    }
    
    else if (country_temp_5[b] == "herzegovina") {
        country_temp[b] = "bosnia and herzegovina"
    }
    
    else if (country_temp_5[b] == "korea") {
        country_temp[b] = "south korea"
    }
    
    else if (country_temp_5[b] == "netherland") {
        country_temp[b] = "netherlands"
    }
    
    else if (country_temp_5[b] == "new guinea") {
        country_temp[b] = "papua new guinea"
    }
    
    else if (country_temp_5[b] == "philippine") {
        country_temp[b] = "philippines"
    }
    
    else if (country_temp_5[b] == "surinam") {
        country_temp[b] = "suriname"
    }
    
    else if (country_temp_5[b] == "uk") {
        country_temp[b] = "united kingdom"
    }
    
    else if (country_temp_5[b] == "urundi") {
        country_temp[b] = "burundi"
    }
    
    else if (grepl("exacavated|excavated", country_temp_5[b])) {
        country_temp[b] = "jordan"
    }
    
    b = b + 1
}


In [13]:
# Create new column in met_df to contain Country_2 data

met_df$Country_2 <- country_temp

## **Met: Nationality**

In [102]:
# Make Strings Lowercase

nationality_temp <- met_df$Nationality

nationality_temp = tolower(nationality_temp)

In [103]:
# Remove Stop Words

stop_words_nationality <- c("active in", "family", "possibly", "probably")

nationality_temp = removeWords(nationality_temp, stop_words_nationality) 

In [104]:
# Split Strings via Special Character Delimiters

nationality_temp_2 <- strsplit(nationality_temp, split="[|,/?:[(]+")

m = 1

while (m < length(nationality_temp)) {
       
    if ((is.na(nationality_temp_2[[m]][1]) | nationality_temp_2[[m]][1] == '') & length(nationality_temp_2[[m]]) == 1) {
        nationality_temp[m] = "unknown"
    }
    
    else {
        nationality_temp[m] = nationality_temp_2[[m]][1]
    }
    
    m = m + 1
}


In [105]:
# Clean strings

nationality_temp = clean_strings(nationality_temp)

In [106]:
# Cardinal Directions & "or" / "and": Round 1

delim_nationality_andor <- function(nat_list, nat_temp) {
    n = 1

    while (n < length(nat_list)) {
        if ((is.na(nat_temp[[n]][1]) | nat_temp[[n]][1] == '') & length(nat_temp[[n]]) == 1) {
            nat_list[n] = nat_temp[[n]][2]
        }

        else {
            nat_list[n] = nat_temp[[n]][1]
        }

        n = n + 1
    }
    
    return(nat_list)
}


nationality_temp_3a <- strsplit(nationality_temp, split=" or ")

nationality_temp = delim_country_andor(nationality_temp, nationality_temp_3a)

nationality_temp_3b = strsplit(nationality_temp, split=" and ")

nationality_temp = delim_country_andor(nationality_temp, nationality_temp_3b)

nationality_temp_3c = strsplit(nationality_temp, split=" born ")

nationality_temp = delim_country_andor(nationality_temp, nationality_temp_3c)


In [107]:
# Cardinal Directions: Round 2

nationality_temp_4 <- strsplit(nationality_temp, split=" ")

o = 1

while (o < length(nationality_temp)) {

    if (length(nationality_temp_4[[o]]) > 1) {
        
        if (nationality_temp_4[[o]][1] == "south" & nationality_temp_4[[o]][2] == "african") {
            
            nationality_temp[o] == "south african"
        }
        
        else if (nationality_temp_4[[o]][1] == "western" & nationality_temp_4[[o]][2] == "saharan") {
            
            nationality_temp[o] == "western saharan"
        }
        
        else if (nationality_temp_4[[o]][1] == "south" & nationality_temp_4[[o]][2] == "korean") {
            
            nationality_temp[o] == "south korean"
        }
        
        else if (nationality_temp_4[[o]][1] == "south" & nationality_temp_4[[o]][2] == "sudan") {
            
            nationality_temp[o] == "south sudan"
        }
        
        else if (nationality_temp_4[[o]][1] == "south" & nationality_temp_4[[o]][2] == "georgia") {
            
            nationality_temp[o] == "south georgian"
        }
        
        else {
            
            z <- nationality_temp_4[[o]][!grepl("north|east|south|west", nationality_temp_4[[o]])]
            nationality_temp[o] = paste(z, collapse=' ')
        }
    }
    
    o = o + 1
}

In [108]:
# Standardize Nationality Spellings

nationality_temp_5 = str_replace_all(nationality_temp, c("franco " = "",
                                                          "american german|british american|chiricahua apache native american|antelope valley washoe|first nations|french american|german american|iranian american|japanese american|mexican american|new york|native american" = "american",
                                                          "armenian iranian|persian" = "iranian",
                                                          "asmat" = "papua new guinea",
                                                          "austro hungarian" = "hungarian",
                                                          "byzantine" = "greek",
                                                          "bohemian|czechoslovakian" = "czech",
                                                          "algonquin|haida|inuit" = "canadian",
                                                          "danish icelandic" = "icelandic",
                                                          "denmark" = "danish",
                                                          "dem republic of the congo|democratic republic of congo|democratic republic of the congo" = "congolese",
                                                          "ecuadorian" = "ecuadorean",
                                                          "alsatian|france by 1894|fench|france|france by 1894|french 1866 1932|paris" = "french",
                                                          "finish" = "finnish",
                                                          "greenlandish" = "greenlandic",
                                                          "emilian|florence|french italian|italy|lombard|naples|siena|sienese|tuscany|venetian|veronese" = "italian",
                                                          "burgundian|french dutch|netherlands|nethrlandish|netherlandish|netherland" = "dutch",
                                                          "belgium|brabant|flemish" = "belgian",
                                                          "aachen|austrian german|bremen|geman|germany|hannoverian|hannover|hanoverian|prussian|upper rhine" = "german",
                                                          "khitan|manchu" = "chinese",
                                                          "iraqi kurdish" = "iraqi",
                                                          "ottoman" = "turkish",
                                                          "london|english|united kingdom|scottish|welsh" = "british",
                                                          "manufacturer" = "unknown",
                                                          "norway" = "norwegian",
                                                          "papua new guinea" = "papuan",
                                                          "polish lithunanian" = "lithuanian",
                                                          "rumanian" = "romanian",
                                                          "serbo croat" = "croatian",
                                                          "ukranian" = "ukrainian"
                                                         ))


# Standardize names shorter than their replacements:
nationality_temp = nationality_temp_5

b <- 1
while (b < length(nationality_temp)) {
    if (is.na(nationality_temp_5[b]) | nationality_temp_5[b] == "") {
        nationality_temp[b] = "unknown"
    }
    
    else if (nationality_temp_5[b] == "africa") {
        nationality_temp[b] = "african"
    }
    
    else if (nationality_temp_5[b] == "america" | nationality_temp_5[b] == "us" | nationality_temp_5[b] == "u s" | nationality_temp_5[b] == "usa" | nationality_temp_5[b] == "united states") {
        nationality_temp[b] = "american"
    }
    
    
    else if (nationality_temp_5[b] == "austria") {
        nationality_temp[b] = "austrian"
    }
    
    else if (nationality_temp_5[b] == "cameroon") {
        nationality_temp[b] = "cameroonian"
    }
    
    else if (nationality_temp_5[b] == "canada") {
        nationality_temp[b] = "canadian"
    }
    
    else if (nationality_temp_5[b] == "congo") {
        nationality_temp[b] = "congolese"
    }
    
    else if (nationality_temp_5[b] == "gabon") {
        nationality_temp[b] = "gabonese"
    }
    
    else if (nationality_temp_5[b] == "iran") {
        nationality_temp[b] = "iranian"
    }
    
    else if (nationality_temp_5[b] == "japan") {
        nationality_temp[b] = "japanese"
    }
    
    else if (nationality_temp_5[b] == "kashmir") {
        nationality_temp[b] = "indian"
    }
    
    else if (nationality_temp_5[b] == "nigeria") {
        nationality_temp[b] = "nigerian"
    }
    
    else if (nationality_temp_5[b] == "roman") {
        nationality_temp[b] = "italian"
    }
    
    else if (nationality_temp_5[b] == "sierra leone") {
        nationality_temp[b] = "sierra leonean"
    }
    
    else if (nationality_temp_5[b] == "tajikistan") {
        nationality_temp[b] = "tajikistani"
    }
    
    else if (nationality_temp_5[b] == "uganda") {
        nationality_temp[b] = "ugandan"
    }
    
    b = b + 1
}


In [109]:
# Clean strings

nationality_temp = clean_strings(nationality_temp)

In [110]:
# Create new column 'Country_2'

met_df$Nationality_2 <- nationality_temp

## **Countries Linking Table: Country**

In [27]:
# Clean strings and convert to lowercase

country_link <- countries$Country
country_link = tolower(country_link)
country_link = clean_strings(country_link)

In [28]:
countries$Country_2 <- country_link

In [112]:
# Print met_df country values that are / are not in the 'countries' linking table

lcn <- c(names(table(countries$Country_2)))
cn <- c(names(table(met_df$Country_2)))

print("--- --- --- --- ---")
print("IN LINKING TABLE:")
print("--- --- --- --- ---")
for (c in cn) {
    if ((c %in% lcn)) {
        print(c)
    }
}

print("")
print("")
print("--- --- --- --- ---")
print("NOT IN LINKING TABLE:")
print("--- --- --- --- ---")
for (c in cn) {
    if (!(c %in% lcn)) {
        print(c)
    }
}

[1] "--- --- --- --- ---"
[1] "IN LINKING TABLE:"
[1] "--- --- --- --- ---"
[1] "afghanistan"
[1] "albania"
[1] "algeria"
[1] "angola"
[1] "argentina"
[1] "armenia"
[1] "australia"
[1] "austria"
[1] "azerbaijan"
[1] "bangladesh"
[1] "belgium"
[1] "benin"
[1] "bermuda"
[1] "bhutan"
[1] "bolivia"
[1] "bosnia and herzegovina"
[1] "brazil"
[1] "bulgaria"
[1] "burkina faso"
[1] "burundi"
[1] "cambodia"
[1] "cameroon"
[1] "canada"
[1] "central african republic"
[1] "chad"
[1] "chile"
[1] "china"
[1] "colombia"
[1] "cook islands"
[1] "costa rica"
[1] "croatia"
[1] "cuba"
[1] "cyprus"
[1] "czech republic"
[1] "democratic republic of congo"
[1] "denmark"
[1] "dominican republic"
[1] "ecuador"
[1] "egypt"
[1] "el salvador"
[1] "equatorial guinea"
[1] "ethiopia"
[1] "fiji"
[1] "finland"
[1] "france"
[1] "french guiana"
[1] "french polynesia"
[1] "gabon"
[1] "gambia"
[1] "georgia"
[1] "germany"
[1] "ghana"
[1] "greece"
[1] "greenland"
[1] "guatemala"
[1] "guinea"
[1] "guinea bissau"
[1] "guyana"
[

In [ ]:
# Working with 'unknown' values. The majority of records do not have accurate 
# location information for their origins. This felt like the end of the road
# for the project, since missing the majority of the location information
# makes drawing accurate conclusions about cultural representation difficult.

# Below, the 'Country_2' column will be used to supplement country data

# However, this data also can speak in other ways (in addition to mapping
# location values that do exist). For example, what types of objects are least and
# most likely to have locations entered? Are items on display more likely to have
# accurate location details? Do other correlations exist, such as credit or period? 

# Update: 1 Sept 2023 -- Decided to make this into a ML project. Can I train a model on existing
# Country & Nationality data to fill in missing values?

# met_df[met_df$Country_2 == 'unknown',]

## **Countries Linking Table: Nationality**

In [30]:
# Clean strings and convert to lowercase

nationality_link <- countries$Nationality
nationality_link = tolower(nationality_link)
nationality_link = clean_strings(nationality_link)

In [31]:
countries$Nationality_2 <- nationality_link

In [111]:
# Print met_df nationality values that are / are not in the 'countries' linking table

lnn <- c(names(table(countries$Nationality_2)))
nn <- c(names(table(met_df$Nationality_2)))

print("--- --- --- --- ---")
print("IN LINKING TABLE:")
print("--- --- --- --- ---")
for (n in nn) {
    if ((n %in% lnn)) {
        print(n)
    }
}

print("")
print("")
print("--- --- --- --- ---")
print("NOT IN LINKING TABLE:")
print("--- --- --- --- ---")
for (n in nn) {
    if (!(n %in% lnn)) {
        print(n)
    }
}

[1] "--- --- --- --- ---"
[1] "IN LINKING TABLE:"
[1] "--- --- --- --- ---"
[1] "albanian"
[1] "algerian"
[1] "american"
[1] "andorran"
[1] "argentinian"
[1] "armenian"
[1] "australian"
[1] "austrian"
[1] "belgian"
[1] "brazilian"
[1] "british"
[1] "bulgarian"
[1] "cameroonian"
[1] "canadian"
[1] "chilean"
[1] "chinese"
[1] "colombian"
[1] "congolese"
[1] "costa rican"
[1] "croatian"
[1] "cuban"
[1] "czech"
[1] "danish"
[1] "dominican"
[1] "dutch"
[1] "ecuadorean"
[1] "egyptian"
[1] "finnish"
[1] "french"
[1] "gabonese"
[1] "georgian"
[1] "german"
[1] "ghanaian"
[1] "greek"
[1] "greenlandic"
[1] "guatemalan"
[1] "hungarian"
[1] "icelandic"
[1] "indian"
[1] "iranian"
[1] "iraqi"
[1] "irish"
[1] "israeli"
[1] "italian"
[1] "japanese"
[1] "korean"
[1] "kuwaiti"
[1] "lebanese"
[1] "lithuanian"
[1] "malaysian"
[1] "malian"
[1] "maltese"
[1] "mexican"
[1] "mongolian"
[1] "new zealander"
[1] "nigerian"
[1] "norwegian"
[1] "pakistani"
[1] "panamanian"
[1] "papuan"
[1] "peruvian"
[1] "polish"
[

# Data Cleaning: Country & Nationality
## Phase 2: Replace 'Unknown' Values via Linking Table

If either (but not both) Country_2 or Nationality_2 in 'met_df' is blank, 
the other value will be matched to the corresponding 
'countries' value, and the missing value in 'met_df' will replace 'unknown.'


### Before this cleaning phase:

* Country_2 has 366365 unknown records out of 448203 total, and Nationality_2 has 252232
* Among unknown Country_2 values, 189009 have known values in the Nationality_2 column 
* Among unknown Nationality_2 values, 71552 have known values in the Country_2 column

### **Step 1: Add missing Country_2 values with existing Nationality_2 values**

In [34]:
# Create df with unknown countries

unknown_cty_df <- met_df[met_df$Country_2 == 'unknown',]

In [35]:
# Create df from rows in unknown_df with known nationalities
# Object_Number will be used to map new Country_2 values back to met_df

unknown_cty_df_compare <- unknown_cty_df[unknown_cty_df$Nationality_2 != 'unknown',]

In [36]:
# A little over half (51.6%) of the unknown country values have a known nationality value

nrow(unknown_cty_df_compare)

[1] 193244

In [37]:
# Create temp dfs of 1) unknown countries in met_df and 2) a linking table based on 'countries' df
# These will be used for a left join on 'Country_2'

nat_name_temp <- unknown_cty_df_compare %>%
    select(Object_Number, Nationality_2)

cty_link_temp <- countries %>%
    select(Country_2, Nationality_2)


In [38]:
# Merge unknown_df_nat with countries on Country_2

cty_nat_id <- merge(x=nat_name_temp, y=cty_link_temp, by='Nationality_2', x.all=TRUE)

In [39]:
# All but 22 records updated

nrow(cty_nat_id)

[1] 193210

In [40]:
# Create new df with only Object_Number & Country_2 columns for use in left join below

cty_new <- cty_nat_id %>%
    select(Object_Number, Country_2) %>%
    rename('Country_3' = 'Country_2')

In [41]:
# Left Join met_df to cty_new to add Country_3 column

met_df2 <- met_df %>% left_join(cty_new,
           by=c('Object_Number'))

### **Step 2: Add missing Country_2 values with existing Country_2 values**

In [42]:
# Create df with unknown nationalities

unknown_nat_df <- met_df[met_df$Nationality_2 == 'unknown',]

In [43]:
# Create df from rows in unknown_df with known countries
# Object_Number will be used to map new Nationality_2 values back to met_df

unknown_nat_df_compare <- unknown_nat_df[unknown_nat_df$Country_2 != 'unknown',]

In [44]:
# Create temp dfs of 1) unknown nationalities in met_df and 2) a linking table based on 'countries' df
# These will be used for a left join on 'Country_2'

cty_name_temp <- unknown_nat_df_compare %>%
    select(Object_Number, Country_2)

nat_link_temp <- countries %>%
    select(Country_2, Nationality_2)

In [45]:
# Merge unknown_df_nat with countries on Country_2

nat_cty_id <- merge(x=cty_name_temp, y=nat_link_temp, by='Country_2', x.all=TRUE)

In [46]:
# 71438 records updated (all but 114)

nrow(nat_cty_id)

[1] 71438

In [47]:
# Create new df with only Object_Number & Nationality_2 columns for use in left join below

nat_new <- nat_cty_id %>%
    select(Object_Number, Nationality_2) %>%
    rename('Nationality_3' = 'Nationality_2')

In [64]:
# Left Join met_df2 to nat_new to add Country_3 column

met_df3 <- met_df2 %>% left_join(nat_new, 
           by=c('Object_Number'))

### **Step 3: Use Department Value if both Country & Nationality 'unknown'**

In [ ]:
# met_df$Country_2 & met_df$Country_2 values BOTH == 'unknown'

# List all Department values in met_df

print(table(met_df$Department))

In [65]:
# Filter by Department: objects missing country and/or nationality values

# American
dept_us_df <- met_df[met_df$Department == 'American Decorative Arts' | met_df$Department == 'American Paintings and Sculpture',]
dept_us_unknown_df <- dept_us_df[dept_us_df$Country_2 == 'unknown' & dept_us_df$Nationality_2 == 'unknown',]

dept_us_unknown_df$Country_3 <- rep(c("united states"), times=nrow(dept_us_unknown_df))
dept_us_unknown_df$Nationality_3 <- rep(c("american"), times=nrow(dept_us_unknown_df))


# Egyptian
dept_eg_df <- met_df[met_df$Department == 'Egyptian Art',]
dept_eg_unknown_df <- dept_eg_df[dept_eg_df$Country_2 == 'unknown' & dept_eg_df$Nationality_2 == 'unknown',]

dept_eg_unknown_df$Country_3 <- rep(c("egypt"), times=nrow(dept_eg_unknown_df))
dept_eg_unknown_df$Nationality_3 <- rep(c("egyptian"), times=nrow(dept_eg_unknown_df))

In [66]:
# Combine the new dfs

dept_known_df <- rbind(dept_us_unknown_df, dept_eg_unknown_df)

In [67]:
# Create temp df for combining country & nationality values with met_df4 below

dept_new <- dept_known_df %>%
    select(Object_Number, Country_3, Nationality_3) 

In [68]:
# Create met_df4 from met_df3

met_df4 <- met_df3

In [69]:
# Combine dept_new df with met_df4

met_df4$Country_3[met_df4$Object_Number %in% dept_new$Object_Number] <- dept_new$Country_3[dept_new$Object_Number %in% met_df4$Object_Number]
met_df4$Nationality_3[met_df4$Object_Number %in% dept_new$Object_Number] <- dept_new$Nationality_3[dept_new$Object_Number %in% met_df4$Object_Number]

Warning message in met_df4$Country_3[met_df4$Object_Number %in% dept_new$Object_Number] <- dept_new$Country_3[dept_new$Object_Number %in% :
“number of items to replace is not a multiple of replacement length”
Warning message in met_df4$Nationality_3[met_df4$Object_Number %in% dept_new$Object_Number] <- dept_new$Nationality_3[dept_new$Object_Number %in% :
“number of items to replace is not a multiple of replacement length”


### **Step 4: Replace 'unknown' values in Country_2 and Nationality_2 Columns**

In [100]:
# Create met_df4 from met_df3

met_df5 <- met_df4

In [101]:
# Set 'unknown' values in Country_2 and Nationality_2 to NA 
# to prepare to coalesce with Country_3 and Nationality_3, respectively

met_df5$Country_2[met_df5$Country_2 == "unknown"] <- NA

met_df5$Nationality_2[met_df5$Nationality_2 == "unknown"] <- NA

In [102]:
# Coalesce Columns

# Country
met_df5$Country_WGL <- coalesce(met_df5$Country_2, met_df5$Country_3)

# Nationality
met_df5$Nationality_WGL <- coalesce(met_df5$Nationality_2, met_df5$Nationality_3)

In [103]:
# Create new df from cleaned Met data

met_clean_df <- met_df5 %>%
    select(Object_Number, Object_Name, Title, 
           Department, Period, Object_Year, 
           # Geography_Type_WGL, Culture_WGL, 
           Nationality_WGL, Country_WGL, Region, Sub_Region, 
           Artist_Name, Medium, Classification, Dimensions, Credit) %>%
    rename(
        # "Geography_Type_WGL" = "Geography_Type", "Culture_WGL" = "Culture", 
           "Country" = "Country_WGL", "Nationality" = "Nationality_WGL")

In [104]:
# Reassign 'unknown' to NA Country and Nationality values (improves tracking and avoids errors)

# Country
met_clean_df$Country[is.na(met_clean_df$Country)] <- "unknown"

# Nationality
met_clean_df$Nationality[is.na(met_clean_df$Nationality)] <- "unknown"

In [62]:
nrow(met_clean_df)

[1] 450323

In [105]:
head(met_clean_df, n=5000)

,Object_Number,Object_Name,Title,Department,Period,Object_Year,Nationality,Country,Region,Sub_Region,Artist_Name,Medium,Classification,Dimensions,Credit
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1979.486.1,Coin,One-dollar Liberty Head Coin,American Decorative Arts,,1853,american,united states,,,James Barton Longacre,Gold,Metal,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979"
2,1980.264.5,Coin,Ten-dollar Liberty Head Coin,American Decorative Arts,,1901,american,united states,,,Christian Gobrecht,Gold,Metal,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980"
3,67.265.9,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"
4,67.265.10,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"
5,67.265.11,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"
6,67.265.12,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"
7,67.265.13,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"
8,67.265.14,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"
9,67.265.15,Coin,Two-and-a-Half Dollar Coin,American Decorative Arts,,1927,american,united states,,,,Gold,Metal,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love, Jr., 1967"


## **Update after Phase 2**:

* met_clean_df: Country has 175717 unknown records out of 450311 total, and Nationality has 177296 
* 190648 records were added to Country (52% increase), and 74936 were added to Nationality (29.7% increase)

### Background: 
* met_df: Country_2 had 366365 unknown records; Country_2 had 252232

### Reflections:
* A mysterious 2108 records added to met_clean_df: total rows for met_clean_df is 450311, while met_df is only 448203
* There is also a 1579-value discrepancy between unknowns in Country and Nationality in met_clean_df

These may be related to country and nationality values in met_df that do not correspond to the countries linking df.

With the majority of country data tracking, can this portion of the dataset be used to train an ML model that can predict the country data for the remaining unknown values? 

Many of these columns contain string data; how could it best be quantified within a training set?

The most obvious identifying feature - the image - is also missing. Using the image would be helpful in predicting unknown country values.

The 'Link_Resource' column is easy to feed into a web scraper; however, some of the Met images online are public domain, while others are not.